In [1]:
import cv2


# out = cv2.VideoWriter('./de_capture2.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (512, 512))
out = cv2.VideoWriter('./de_capture4.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))

cap = cv2.VideoCapture(0)
while True:
    ret, img = cap.read()
    # img1 = img.copy()
    # img1 = img[10:600, 50:600]
    # cv2.rectangle(img1, (50,10),(600,500),(0,255,0),2)
    # img1 = cv2.resize(img1, (512,512))
    if not ret:
        break

    # print(img.shape)
    # cv2.imshow("Image", img1)
    # out.write(img1)
    cv2.imshow("Image", img)
    out.write(img)

    k = cv2.waitKey(1)

    if k==27:
        break

out.release()
cv2.destroyAllWindows()

In [2]:
import cv2
cap = cv2.VideoCapture("../obama_fullhd.mp4")
print("FPS:", int(cap.get(cv2.CAP_PROP_FPS)))

while True:
    ret, img = cap.read()
    if not ret:
        break
    print("Shape: ", img.shape)
    
    cv2.imshow("Image", img1)
    k = cv2.waitKey(0)

    if k==27:
        break

cv2.destroyAllWindows()

FPS: 29
Shape:  (1080, 1920, 3)


In [ ]:
import dlib
import numpy as np
import cv2

import sys
sys.path.insert(0, "../../deepfeatinterp")
import alignface


out = cv2.VideoWriter('./obama_mouth_seemless_clone.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))


model_path = "/home/vuthede/Downloads/shape_predictor_81_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model_path)

def get_landmark(im1):
    det0 = detector(im1, 0)[0]
    shape = predictor(im1, det0)
    landmarks = np.matrix([[p.x, p.y] for p in shape.parts()])
    landmarks = np.array(landmarks)
    return landmarks

def draw_landmark(im, l, color =(0,255,0)):
    im_ = im.copy()
    for i in l:
        cv2.circle(im_, (i[0],i[1]), 3,color, -1)

    return im_

def get_midpoint(p1, p2):
    return (p1+p2)//2

def generate_more_points(p_nose, p_left, p_right):
    p1 = get_midpoint(p_nose, p_left)
    p1_1 = get_midpoint(p_left, p1)
    p1_2 = get_midpoint(p_nose, p1)

    p2 = get_midpoint(p_nose, p_right)
    p2_1 = get_midpoint(p_right, p2)
    p2_2 = get_midpoint(p_nose, p2)

    return [p1_1, p1, p1_2, p2_1, p2, p2_2]


lmkd_ind = list(range(3,14)) + [30]

obama_syn = cv2.VideoCapture("../result2.mp4")
obama_fullhd = cv2.VideoCapture("../obama_fullhd.mp4")

de = cv2.VideoCapture("../cropyeah.mp4")

x1 = 685
y1 = 85
x2 = 1250
y2 = 650

face_d,face_p=alignface.load_face_detector(predictor_path="/home/vuthede/AI/deepfeatinterp/models/shape_predictor_68_face_landmarks.dat")

while True:
    ret, img_obama_syn = obama_syn.read()
    for i in range(3):
        ret1, img_obamahd = obama_fullhd.read()
    ret2, img_de = de.read()

    obama_crop = img_obamahd[y1:y2, x1:x2]
    if not ret or not ret1:
        break

    img_obama_syn = cv2.resize(img_obama_syn, (x2-x1, y2-y1))
    img_de = cv2.resize(img_de, (x2-x1, y2-y1))

    # concat = np.hstack([img_de, img_obama_syn])

    lm1 = get_landmark(img_obama_syn)
    lm2 = get_landmark(obama_crop)
    lm1_full = get_landmark(img_obama_syn)
    lm2_full = get_landmark(obama_crop)
    lm1_extend = generate_more_points(lm1[30], lm1[3], lm1[13])
    lm2_extend = generate_more_points(lm2[30], lm2[3], lm2[13])
    lm1 = np.array(list(lm1[lmkd_ind]) + list(lm1_extend))
    lm2 = np.array(list(lm2[lmkd_ind]) + list(lm2_extend))
    lm1_full = np.array(list(lm1_full) + list(lm1_extend))
    lm2_full = np.array(list(lm2_full) + list(lm2_extend))


    cv2.imwrite("tmp.png", img_obama_syn)
    cv2.imwrite("tmp2.png", obama_crop)

    template,original=alignface.detect_landmarks("tmp.png",face_d,face_p)
    template2,original2=alignface.detect_landmarks("tmp2.png",face_d,face_p)

    # Find tranformation matrix
    # M,loss=alignface.fit_face_landmarks(lm1,lm2,landmarks=list(range(0,18))+[0,10],\
    #                                    scale_landmarks=[0,10],location_landmark=11, image_dims=img_obama_syn.shape[:2])
    
    
    # M,loss=alignface.fit_face_landmarks(lm1_full,lm2_full,landmarks=list(range(87)), image_dims=img_obama_syn.shape[:2])
    M,loss=alignface.fit_face_landmarks(template,template2, image_dims=img_obama_syn.shape[:2])

    print("matrix transform:", M)
    print("Loss transform:", loss)
    warp_image = alignface.warp_to_template(img_obama_syn,M,border_value=(0.5,0.5,0.5),image_dims=img_obama_syn.shape[:2])

    print(lm2)
    print(M.shape)

    #  Warp landmarks
    points = np.hstack([lm1, np.ones((len(lm1),1))])
    lm2_warp = np.matmul(M, points.T).T.astype(int)


    lm3 = get_landmark(warp_image)
    lm3_extend = generate_more_points(lm3[30], lm3[3], lm3[13])
    lm3 = np.array(list(lm3[lmkd_ind]) + list(lm3_extend))


    # Find convex hull
    hull1 = []
    hull2 = []
    hullIndex = cv2.convexHull(lm1, returnPoints = False)          
    for i in range(0, len(hullIndex)):
        hull1.append(lm1[int(hullIndex[i])])
        hull2.append(lm2[int(hullIndex[i])])
    
    hull1 = np.array(hull1)
    hull2 = np.array(hull2)
    # print(hull1)

    mouth_mask = np.zeros(img_obama_syn.shape, img_obama_syn.dtype)
    cv2.fillPoly(mouth_mask, [hull2], (255, 255, 255))

    # cv2.imshow("Mash", mouth_mask)
    r = cv2.boundingRect(np.float32(hull2))    
    center = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))
    clone = cv2.seamlessClone(warp_image, obama_crop, mouth_mask, center, cv2.NORMAL_CLONE)

    obama_crop_copy = obama_crop.copy()
    obama_crop_copy2 = obama_crop.copy()

    
    obama_crop_copy[mouth_mask.astype(bool)] = warp_image[mouth_mask.astype(bool)]
    obama_crop_copy2[mouth_mask.astype(bool)] = img_obama_syn[mouth_mask.astype(bool)]


    concat = np.hstack([img_de, clone])


    warp_image = draw_landmark(draw_landmark(draw_landmark(warp_image, lm2_warp), lm2, (255,0,0)), lm1, (0,0,255))
    cv2.imshow("Obama syn", draw_landmark(draw_landmark(img_obama_syn, lm1), lm1[list(range(0,18))+[0,10]], (0,0,255)))
    cv2.imshow("Obama original crop", draw_landmark(draw_landmark(obama_crop, lm2), lm2[list(range(0,18))+[0,10]], (255,0,0)))
    cv2.imshow("Obama Clone", clone)
    cv2.imshow("Obama concat", concat)
    cv2.imshow("Warp Image:", warp_image)
    cv2.imshow("Mouth mask:", mouth_mask)
    cv2.imshow("Overlay_warp:", obama_crop_copy)
    cv2.imshow("Overlay syn:", obama_crop_copy2)







    k = cv2.waitKey(0)

    if k==27:
        break

cv2.destroyAllWindows()


